In [1]:
#ds=xr.Dataset({'qc':qcXR,'qr':qrXR,'qi':qiXR,'qs':qsXR,'qg':qgXR,'qns':qnsXR,'qng':qngXR,'qnr':qnrXR,'qni':qniXR,'qv':qvXR,     'press':pressXR,'temp':tempXR,'dz':dzXR,'qc_tend':qc_tendXR,'qr_tend':qr_tendXR,'qi_tend':qi_tendXR,'qs_tend':qs_tendXR,'qg_tend':qg_tendXR,'qns_tend':qns_tendXR,'qng_tend':qng_tendXR,'qnr_tend':qnr_tendXR,'qni_tend':qni_tendXR,'qv_tend':qv_tendXR,'temp_tend':temp_tendXR})

from netCDF4 import Dataset
import netCDF4 as nc
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.colors as colors

with nc.Dataset("trainingData.nc") as fh:
    qc=fh.variables['qc'][:]
    qr=fh.variables['qr'][:]
    qi=fh.variables['qi'][:]
    qs=fh.variables['qs'][:]
    qg=fh.variables['qg'][:]
    qns=fh.variables['qns'][:]
    qng=fh.variables['qng'][:]
    qnr=fh.variables['qnr'][:]
    qni=fh.variables['qni'][:]
    qv=fh.variables['qv'][:]
    press=fh.variables['press'][:]
    temp=fh.variables['temp'][:]
    dz=fh.variables['dz'][:]
    qc_tend=fh.variables['qc_tend'][:]
    qr_tend=fh.variables['qr_tend'][:]
    qi_tend=fh.variables['qi_tend'][:]
    qs_tend=fh.variables['qs_tend'][:]
    qg_tend=fh.variables['qg_tend'][:]
    qns_tend=fh.variables['qns_tend'][:]
    qng_tend=fh.variables['qng_tend'][:]
    qnr_tend=fh.variables['qnr_tend'][:]
    qni_tend=fh.variables['qni_tend'][:]
    qv_tend=fh.variables['qv_tend'][:]
    temp_tend=fh.variables['temp_tend'][:]
    nt,nz=qc.shape
    

In [2]:
#scale variables using the standard scaler
from sklearn.preprocessing import StandardScaler
scaler=StandardScaler()

inputVars=["qc","qr","qi","qs","qg","qv","press","temp","dz"]
outputVars=["qc_tend","qr_tend","qi_tend","qs_tend","qg_tend","qv_tend","temp_tend"]
inputData={"qc":qc,"qr":qr,"qi":qi,"qs":qs,"qg":qg,"qns":qns,"qng":qng,"qnr":qnr,"qni":qni,"qv":qv,"press":press,"temp":temp,"dz":dz}
outputData={"qc_tend":qc_tend,"qr_tend":qr_tend,"qi_tend":qi_tend,"qs_tend":qs_tend,"qg_tend":qg_tend,"qns_tend":qns_tend,"qng_tend":qng_tend,"qnr_tend":qnr_tend,"qni_tend":qni_tend,"qv_tend":qv_tend,"temp_tend":temp_tend}
inputDataScaled={}
outputDataScaled={}
inputScalers={}
outputScalers={}
for var in inputVars:
    vmean=inputData[var].mean()
    vstd=inputData[var].std()
    inputDataScaled[var]=(inputData[var]-vmean)/vstd
    inputScalers[var]=(vmean,vstd)
for var in outputVars:
    vmean=outputData[var].mean()
    vstd=outputData[var].std()
    outputDataScaled[var]=(outputData[var]-vmean)/vstd
    outputScalers[var]=(vmean,vstd)

In [3]:
inputData=np.array([inputDataScaled[var] for var in inputVars]).T
outputData=np.array([outputDataScaled[var] for var in outputVars]).T
inputData=np.swapaxes(inputData,0,1)
outputData=np.swapaxes(outputData,0,1)
print(inputData.shape,outputData.shape)


(684284, 72, 9) (684284, 72, 7)


In [4]:
print(dz[0,0:64].sum())

15450.283


In [5]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(inputData,outputData,test_size=0.4,random_state=42)

In [6]:
print(X_train.shape,X_test.shape,y_train.shape,y_test.shape)

(410570, 72, 9) (273714, 72, 9) (410570, 72, 7) (273714, 72, 7)


In [7]:
import pickle
pickle.dump(inputScalers,open("inputScalers_1M_cell.pkl","wb"))
pickle.dump(outputScalers,open("outputScalers_1M_cell.pkl","wb"))

In [49]:
#define dense model
from keras.models import Sequential
from keras.layers import Dense,Activation,Dropout
from keras.optimizers import SGD
from keras.callbacks import EarlyStopping
from keras import regularizers
from tensorflow.keras.layers import Input, Dense
import tensorflow as tf

model_gc0=Sequential()
model_gc0.add(Dense(64,input_dim=9,activation='relu'))
#model_gc.add(Dense(64,activation='relu'))
model_gc0.add(Dense(7,activation='linear'))

from keras.models import load_model
model_gc0=load_model("unet1D_MP_model_1M_gc.h5")

def create1Dmodel(input_shape):
    input_data = Input(shape=input_shape)
    out=[]
    for i in range(72):
        out.append(model_gc0(input_data[:,i,:]))
    main_model_output = tf.stack(out,axis=1)
    model = tf.keras.Model(inputs=input_data, outputs=main_model_output)
    return model


In [50]:
model=create1Dmodel((72,9))
model.summary()
test=model(X_train[0:1,:72,:])

Model: "model_6"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_7 (InputLayer)        [(None, 72, 9)]              0         []                            
                                                                                                  
 tf.__operators__.getitem_2  (None, 9)                    0         ['input_7[0][0]']             
 16 (SlicingOpLambda)                                                                             
                                                                                                  
 tf.__operators__.getitem_2  (None, 9)                    0         ['input_7[0][0]']             
 17 (SlicingOpLambda)                                                                             
                                                                                            

In [15]:
print(test.shape)

(1, 72, 7)


In [51]:
model.compile(optimizer='adam',loss='mse',metrics=['mse'])
nt,nz,nf=X_train.shape
nt,nz,nout=y_train.shape
nt2,nz,nf=X_test.shape
nt2,nz,nout=y_test.shape
#X_train1=X_train.reshape(nt*nz,nf)
#y_train1=y_train.reshape(nt*nz,nout)
#X_test1=X_test.reshape(nt2*nz,nf)
#y_test1=y_test.reshape(nt2*nz,nout)
hist=model.fit(X_train[:,:72,:],y_train[:,:72,:],epochs=1,batch_size=32,\
                  validation_data=(X_test[:,:72,:],y_test[:,:72,:]))
#y_=model.predict(X_train[0:1,:,:])
#print(y_.shape)

12831/12831 [==============================] - 74s 5ms/step - loss: 0.1554 - mse: 0.1554 - val_loss: 0.1334 - val_mse: 0.1334


In [35]:
hist=model.fit(X_train[:,:36,:],y_train[:,:36,:],epochs=25,batch_size=32,\
                  validation_data=(X_test[:,:36,:],y_test[:,:36,:]))

Epoch 1/25
12831/12831 [==============================] - 26s 2ms/step - loss: 0.4127 - mse: 0.4127 - val_loss: 0.3864 - val_mse: 0.3864
Epoch 2/25
12831/12831 [==============================] - 26s 2ms/step - loss: 0.4046 - mse: 0.4046 - val_loss: 0.3693 - val_mse: 0.3693
Epoch 3/25
12831/12831 [==============================] - 26s 2ms/step - loss: 0.3997 - mse: 0.3997 - val_loss: 0.3621 - val_mse: 0.3621
Epoch 4/25
12831/12831 [==============================] - 25s 2ms/step - loss: 0.3942 - mse: 0.3942 - val_loss: 0.3562 - val_mse: 0.3562
Epoch 5/25
12831/12831 [==============================] - 25s 2ms/step - loss: 0.3903 - mse: 0.3903 - val_loss: 0.3524 - val_mse: 0.3524
Epoch 6/25
12831/12831 [==============================] - 26s 2ms/step - loss: 0.3865 - mse: 0.3865 - val_loss: 0.3532 - val_mse: 0.3532
Epoch 7/25
12831/12831 [==============================] - 26s 2ms/step - loss: 0.3834 - mse: 0.3834 - val_loss: 0.3515 - val_mse: 0.3515
Epoch 8/25
12831/12831 [=================

In [55]:
model.save("unet1D_MP_model_1M.h5")


/Users/mgrecu/miniconda3/lib/python3.9/site-packages/keras/src/engine/training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [58]:
#load model
from keras.models import load_model
model=load_model("unet1D_MP_model_1M.h5")

In [59]:
import time
t0=time.time()
y_=model.predict(X_test[:,:72,:])
t1=time.time()
print("time to predict=",t1-t0)

8554/8554 [==============================] - 9s 944us/step
time to predict= 9.916295051574707


In [60]:
print(y_.shape[0]/(189*202))
for i,var in enumerate(outputVars):
    print(var,np.corrcoef(y_[:,:,i].flatten(),y_test[:,:72,i].flatten())[0,1])

7.16941694169417
qc_tend 0.9853176310534987
qr_tend 0.9363633691455631
qi_tend 0.9309581459545476
qs_tend 0.9822648826039024
qg_tend 0.8102435513407845
qv_tend 0.9052725888480303
temp_tend 0.9479943231125603


In [ ]:
#import knn regressor
from sklearn.neighbors import KNeighborsRegressor
knn=KNeighborsRegressor(n_neighbors=5,weights='distance',n_jobs=4)
knn.fit(X_train1,y_train1)


In [ ]:
import faiss
d=X_train1.shape[1]
index = faiss.IndexFlatL2(d)   # build the index
index.add(X_train1.astype(np.float32))                  # add vectors to the index
D,I=index.search(X_test1.astype(np.float32),5)     # actual search